<a href="https://colab.research.google.com/github/JONNY-ME/Microsoft-Rice-Disease-Classification-Challenge/blob/main/fastai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Setup


In [ ]:
!pip install --upgrade -q fastai
!pip install timm -q
!pip install albumentations==0.4.6 -q
!pip install transformers -q

     |████████████████████████████████| 509 kB 7.0 MB/s 
     |████████████████████████████████| 117 kB 8.1 MB/s 
     |████████████████████████████████| 4.7 MB 6.7 MB/s 
     |████████████████████████████████| 596 kB 66.7 MB/s 
     |████████████████████████████████| 101 kB 13.4 MB/s 
     |████████████████████████████████| 6.6 MB 55.1 MB/s 


In [ ]:
%%time
!unzip -o -q '/content/drive/MyDrive/Microsoft-rice/Images.zip' -d "/content/Images"

CPU times: user 114 ms, sys: 25.9 ms, total: 140 ms
Wall time: 18.6 s


In [ ]:
from fastai.vision.all import *
import timm
import warnings 
import albumentations as A
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import StratifiedKFold

warnings.filterwarnings('ignore')
set_seed(21, reproducible=True)

# Load The Data


In [ ]:
# Load the provided training data
path = "/content/drive/MyDrive/Microsoft-rice/"
train = pd.read_csv(path+'Train.csv')
test = pd.read_csv(path+'Test.csv')
print(train.shape)
train.head()

(5340, 2)


,Image_id,Label
0,id_004wknd7qd.jpg,blast
1,id_004wknd7qd_rgn.jpg,blast
2,id_005sitfgr2.jpg,brown
3,id_005sitfgr2_rgn.jpg,brown
4,id_00stp9t6m6.jpg,blast


In [ ]:
IMG_HEIGHT = IMG_WIDTH = 224
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)
INCEPTION_MEAN = INCEPTION_STD = (0.5, 0.5, 0.5)


class AlbumentationsTransform(RandTransform):
    "A transform handler for multiple `Albumentation` transforms"
    split_idx, order = None, 2
    def __init__(self, train_aug, valid_aug): store_attr()
    
    def before_call(self, b, split_idx):
        self.idx = split_idx
    
    def encodes(self, img: PILImage):
        if self.idx == 0:
            aug_img = self.train_aug(image=np.array(img))['image']
        else:
            aug_img = self.valid_aug(image=np.array(img))['image']
        
        return aug_img


def get_train_transforms(mean_std):
    augmentations = [
        A.HorizontalFlip(p=.5),
        A.VerticalFlip(p=.5),
        A.RandomRotate90(p=.5),
        A.ImageCompression(quality_lower=99, quality_upper=100),
        A.ShiftScaleRotate(
            shift_limit=0.2, scale_limit=0.2, 
            rotate_limit=45, border_mode=0, p=.5
        ),
        A.Resize(IMG_HEIGHT, IMG_WIDTH),
        A.Cutout(
            max_h_size=int(IMG_HEIGHT*0.4),
            max_w_size=int(IMG_WIDTH*0.4),
            num_holes=1,
            p=.75,
        ),
    ]
    if mean_std=='imagenet':
        augmentations.append(A.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD))
    elif mean_std=='inception':
        augmentations.append(A.Normalize(mean=INCEPTION_MEAN, std=INCEPTION_STD))
    else:
        augmentations.append(A.Normalize(mean=0, std=1))

    augmentations.append(ToTensorV2())
    return A.Compose(augmentations)


def get_valid_transforms(mean_std):
    augmentations = [A.Resize(IMG_HEIGHT, IMG_WIDTH)]
    if mean_std=='imagenet':
        augmentations.append(A.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD))
    elif mean_std=='inception':
        augmentations.append(A.Normalize(mean=INCEPTION_MEAN, std=INCEPTION_STD))
    else:
        augmentations.append(A.Normalize(mean=0, std=1))

    augmentations.append(ToTensorV2())
    return A.Compose(augmentations)

def get_item_tfms(mean_std='imagenet'):
    return [AlbumentationsTransform(get_train_transforms(mean_std), get_valid_transforms(mean_std))]

In [ ]:
Train = train.copy()
Test = test.copy()

# drop the RGN images
Train = Train[Train.index%2 == 0].reset_index(drop=True)
Test = Test[Test.index%2 == 0].reset_index(drop=True)

# encode the target
label_map = dict(zip(Train.Label.unique(), range(Train.Label.nunique())))
Train.Label = Train.Label.map(label_map)

# add image paths
Train['image_path'] = '/content/Images/' + Train.Image_id
Test['image_path'] = '/content/Images/' + Test.Image_id

Train = Train[['image_path', 'Label']]
Test = Test[['image_path']]


skf = StratifiedKFold(10, shuffle=True, random_state=21)
X = Train.drop(columns='Label')
y = Train.Label

for fold, (_, valid_index) in enumerate(skf.split(X, y)):
  Train.loc[valid_index, "fold"] = fold

Train.fold = Train.fold.astype(int)

Train.head(2)

,image_path,Label,fold
0,/content/Images/id_004wknd7qd.jpg,0,6
1,/content/Images/id_005sitfgr2.jpg,1,4


In [ ]:
class Head(nn.Module):
    def __init__(self, in_features, out_features):
        super(Head, self).__init__()
        self.head = nn.Linear(in_features=in_features, out_features=out_features)

    def forward(self, x):
        return self.head(x)


class Net(nn.Module):
    def __init__(
        self,
        base_model="resnet50",
        pretrained=True,
        checkpoint_path=None,
        num_classes=3,
    ):
        super(Net, self).__init__()

        self.backbone = timm.create_model(
            base_model, pretrained=pretrained, checkpoint_path=checkpoint_path
        )
        in_features = self.backbone.get_classifier().in_features
        self.backbone.reset_classifier(num_classes=0, global_pool="avg")
        self.neck = Head(in_features=in_features, out_features=in_features)
        self.head = Head(in_features=in_features, out_features=num_classes)

    def forward(self, x):
        x = self.backbone(x)
        x = self.neck(x)
        x = self.head(x)
        x = F.log_softmax(x, dim=1)
        return x


In [ ]:
import gc
import inspect
from fastai.metrics import accuracy
from fastai.optimizer import OptimWrapper
from torch import optim
from functools import partial
from transformers import AdamW


def free_memory(to_delete: list):
    calling_namespace = inspect.currentframe().f_back

    for _var in to_delete:
        calling_namespace.f_locals.pop(_var, None)
        gc.collect()
        torch.cuda.empty_cache()

def get_dls(fold=0, mean_std='imagenet'):
    clas_block = DataBlock(blocks=(ImageBlock, CategoryBlock),
                        splitter=MaskSplitter(Train.fold == fold),
                        get_x=ColReader("image_path"),
                        get_y=ColReader("Label"),
                        item_tfms=get_item_tfms(mean_std),
                        )
    dls = clas_block.dataloaders(Train, bs=32)
    dls.rng.seed(21)

    return dls

In [ ]:
predictions = []
for fold in [1]:
    print('*'*25+f"Fold {fold}"+'*'*25)
    m_name = f'fold-{fold}'
    dls = get_dls(fold, 'imagenet')
    learn = Learner(dls, Net(base_model='convnext_tiny'), 
                    loss_func=CrossEntropyLossFlat(), metrics=accuracy,
                    opt_func=partial(OptimWrapper, opt=optim.Adam), 
                    cbs=[SaveModelCallback(reset_on_fit=False, fname=m_name)]
            )
    
    learn.fit_one_cycle(20, 2e-4)
    learn.save(m_name+'_stage-1')
    learn = learn.load(m_name)
    learn.freeze_to(-5)
    learn.fit_one_cycle(4, 2e-4)
    learn.save(m_name+'_stage-2');


    model = learn.load(m_name)
    preds, _ = model.tta(dl=dls.test_dl(Test), n=4)
    predictions.append(preds)

    free_memory([model, learn, preds, dls])


*************************Fold 1*************************


Downloading: "https://dl.fbaipublicfiles.com/convnext/convnext_tiny_1k_224_ema.pth" to /root/.cache/torch/hub/checkpoints/convnext_tiny_1k_224_ema.pth


epoch,train_loss,valid_loss,accuracy,time
0,0.927108,0.805465,0.689139,01:27
1,0.638806,0.350234,0.880150,01:24
2,0.407865,0.252218,0.917603,01:24
3,0.301463,0.199915,0.928839,01:25
4,0.238749,0.192024,0.932584,01:24
5,0.195048,0.152664,0.955056,01:24
6,0.139396,0.131868,0.943820,01:24
7,0.108690,0.119111,0.955056,01:24
8,0.077315,0.066210,0.981273,01:24
9,0.060478,0.103776,0.962547,01:24


Better model found at epoch 0 with valid_loss value: 0.8054649829864502.
Better model found at epoch 1 with valid_loss value: 0.3502342998981476.
Better model found at epoch 2 with valid_loss value: 0.25221753120422363.
Better model found at epoch 3 with valid_loss value: 0.19991493225097656.
Better model found at epoch 4 with valid_loss value: 0.19202442467212677.
Better model found at epoch 5 with valid_loss value: 0.1526644229888916.
Better model found at epoch 6 with valid_loss value: 0.1318676769733429.
Better model found at epoch 7 with valid_loss value: 0.11911146342754364.
Better model found at epoch 8 with valid_loss value: 0.06621047854423523.
Better model found at epoch 11 with valid_loss value: 0.0661102831363678.
Better model found at epoch 12 with valid_loss value: 0.052768491208553314.
Better model found at epoch 15 with valid_loss value: 0.05145692452788353.


epoch,train_loss,valid_loss,accuracy,time
0,0.008647,0.060012,0.988764,00:43
1,0.006426,0.059763,0.988764,00:44
2,0.005106,0.062523,0.988764,00:43
3,0.005558,0.065060,0.988764,00:44


In [ ]:
for fold in [1]:
    print('*'*25+f"Fold {fold}"+'*'*25)
    m_name = f'vfold-{fold}'
    dls = get_dls(fold, 'inception')
    learn = Learner(dls, Net(base_model='vit_tiny_patch16_224'), 
                    loss_func=CrossEntropyLossFlat(), metrics=accuracy,
                    opt_func=partial(OptimWrapper, opt=optim.Adam), 
                    cbs=[SaveModelCallback(reset_on_fit=False, fname=m_name)]
            )
    
    learn.fit_one_cycle(20, 2e-4)
    learn.save(m_name+'_stage-1')
    learn = learn.load(m_name)
    learn.freeze_to(-5)
    learn.fit_one_cycle(4, 2e-6)
    learn.save(m_name+'_stage-2');

    model = learn.load(m_name)
    preds, _ = model.tta(dl=dls.test_dl(Test), n=4)
    predictions.append(preds)

    free_memory([model, learn, preds, dls])


*************************Fold 1*************************


epoch,train_loss,valid_loss,accuracy,time
0,0.859827,0.674778,0.681648,00:31
1,0.563294,0.470469,0.823970,00:30
2,0.435966,0.247381,0.906367,00:29
3,0.396185,0.340401,0.861423,00:30
4,0.321884,0.351864,0.872659,00:30
5,0.267657,0.190567,0.932584,00:29
6,0.232450,0.184857,0.947566,00:29
7,0.203562,0.190684,0.921348,00:31
8,0.155876,0.142126,0.947566,00:29
9,0.146283,0.164414,0.947566,00:29


Better model found at epoch 0 with valid_loss value: 0.674778401851654.
Better model found at epoch 1 with valid_loss value: 0.47046932578086853.
Better model found at epoch 2 with valid_loss value: 0.24738053977489471.
Better model found at epoch 5 with valid_loss value: 0.19056667387485504.
Better model found at epoch 6 with valid_loss value: 0.18485680222511292.
Better model found at epoch 8 with valid_loss value: 0.14212560653686523.
Better model found at epoch 10 with valid_loss value: 0.1039305329322815.
Better model found at epoch 12 with valid_loss value: 0.09223298728466034.
Better model found at epoch 13 with valid_loss value: 0.08632835745811462.
Better model found at epoch 14 with valid_loss value: 0.07663137465715408.
Better model found at epoch 15 with valid_loss value: 0.06594683229923248.
Better model found at epoch 16 with valid_loss value: 0.06087595969438553.
Better model found at epoch 18 with valid_loss value: 0.059532761573791504.
Better model found at epoch 19 wi

epoch,train_loss,valid_loss,accuracy,time
0,0.015071,0.059468,0.985019,00:29
1,0.013083,0.059621,0.985019,00:29
2,0.015567,0.059742,0.985019,00:30
3,0.014900,0.059763,0.985019,00:29


In [ ]:
for fold in [1]:
    print('*'*25+f"Fold {fold}"+'*'*25)
    m_name = f'sfold-{fold}'
    dls = get_dls(fold, 'imagenet')
    learn = Learner(dls, Net(base_model='swin_tiny_patch4_window7_224'), 
                    loss_func=CrossEntropyLossFlat(), metrics=accuracy,
                    opt_func=partial(OptimWrapper, opt=optim.Adam), 
                    cbs=[SaveModelCallback(reset_on_fit=False, fname=m_name)]
            )
    
    learn.fit_one_cycle(20, 2e-4)
    learn.save(m_name+'_stage-1')
    learn = learn.load(m_name)
    learn.freeze_to(-5)
    learn.fit_one_cycle(4, 2e-6)
    learn.save(m_name+'_stage-2');

    model = learn.load(m_name)
    preds, _ = model.tta(dl=dls.test_dl(Test), n=4)
    predictions.append(preds)

    free_memory([model, learn, preds, dls])


*************************Fold 1*************************


Downloading: "https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_tiny_patch4_window7_224.pth" to /root/.cache/torch/hub/checkpoints/swin_tiny_patch4_window7_224.pth


epoch,train_loss,valid_loss,accuracy,time
0,0.790986,0.522534,0.794007,00:39
1,0.512871,0.234209,0.917603,00:40
2,0.404622,0.234607,0.906367,00:40
3,0.372338,0.270817,0.902622,00:40
4,0.316115,0.243978,0.917603,00:39
5,0.287027,0.134401,0.951311,00:39
6,0.225006,0.174916,0.928839,00:40
7,0.175126,0.177534,0.932584,00:39
8,0.152256,0.130789,0.943820,00:40
9,0.151982,0.158596,0.951311,00:39


Better model found at epoch 0 with valid_loss value: 0.5225338339805603.
Better model found at epoch 1 with valid_loss value: 0.234208881855011.
Better model found at epoch 5 with valid_loss value: 0.13440115749835968.
Better model found at epoch 8 with valid_loss value: 0.1307893842458725.
Better model found at epoch 10 with valid_loss value: 0.07323650270700455.
Better model found at epoch 11 with valid_loss value: 0.060170069336891174.
Better model found at epoch 14 with valid_loss value: 0.05276849865913391.
Better model found at epoch 15 with valid_loss value: 0.03996825963258743.
Better model found at epoch 18 with valid_loss value: 0.039957381784915924.


epoch,train_loss,valid_loss,accuracy,time
0,0.011651,0.040127,0.988764,00:39
1,0.011043,0.040600,0.988764,00:36
2,0.013800,0.040758,0.988764,00:37
3,0.013635,0.040786,0.988764,00:37


In [33]:
import numpy as np
pred = np.mean([i.numpy() for j, i in enumerate(predictions[-1:])], axis=0)

In [34]:
test_df = test[test.index % 2 == 0].reset_index(drop=True)
submission = pd.DataFrame({'Image_id': test_df['Image_id']})

In [35]:
submission[['blast', 'brown', 'healthy']] = pred
submission

,Image_id,blast,brown,healthy
0,id_00vl5wvxq3.jpg,0.999983,0.000014,2.654768e-06
1,id_01hu05mtch.jpg,0.000029,0.999956,1.458189e-05
2,id_030ln10ewn.jpg,0.967794,0.028506,3.700477e-03
3,id_03z57m8xht.jpg,0.999994,0.000002,4.090999e-06
4,id_04ngep1w4b.jpg,0.906756,0.069082,2.416206e-02
...,...,...,...,...
1140,id_zrdlgjrq3r.jpg,0.000035,0.011876,9.880890e-01
1141,id_zsfayxwipp.jpg,0.000087,0.979384,2.052931e-02
1142,id_ztvp2l9k3h.jpg,0.999331,0.000666,2.980699e-06
1143,id_zwwcma7hlt.jpg,0.999992,0.000007,9.329779e-07


In [36]:
submission.to_csv("/content/fastai.csv", index=False)